# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
# import libraries
import sys
import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report


from sklearn.ensemble import RandomForestClassifier


import pickle

nltk.download(['wordnet', 'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sander.berkers\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sander.berkers\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sander.berkers\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [76]:
# load data from database
engine = create_engine("sqlite:///../data/DisasterResponse.db")
df = pd.read_sql_table("tbl_MessagesCategories", engine)
X = df.message
y = df.iloc[:, 5:]

In [73]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   related                 26216 non-null  int64
 1   request                 26216 non-null  int64
 2   offer                   26216 non-null  int64
 3   aid_related             26216 non-null  int64
 4   medical_help            26216 non-null  int64
 5   medical_products        26216 non-null  int64
 6   search_and_rescue       26216 non-null  int64
 7   security                26216 non-null  int64
 8   military                26216 non-null  int64
 9   child_alone             26216 non-null  int64
 10  water                   26216 non-null  int64
 11  food                    26216 non-null  int64
 12  shelter                 26216 non-null  int64
 13  clothing                26216 non-null  int64
 14  money                   26216 non-null  int64
 15  missing_people     

In [74]:
y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [48]:
def tokenize(text):

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # detect and remove urls from text.
    found_urls = re.findall(url_regex, text)

    for url in found_urls:
        text = text.replace(url, 'weblink_placeholder')

    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words]

    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [49]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [55]:
# vect = CountVectorizer(tokenizer=tokenize)
# tfidf = TfidfTransformer()
# clf = RandomForestClassifier()

# X_train_counts = vect.fit_transform(X_train)
# X_train_tfidf = tfidf.fit_transform(X_train_counts)

# clf.fit(X_train_tfidf, y_train)

In [54]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000019913B78B80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred.shape

In [63]:
for i in range(36):
    print("=======================",y_test.columns[i],"======================")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

======================= related ======================
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      1203
           1       0.85      0.95      0.90      4005
           2       0.33      0.39      0.36        36

    accuracy                           0.83      5244
   macro avg       0.64      0.59      0.60      5244
weighted avg       0.82      0.83      0.81      5244

======================= request ======================
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4375
           1       0.80      0.49      0.61       869

    accuracy                           0.90      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.90      0.89      5244

======================= offer ======================
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0

C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4523
           1       0.48      0.03      0.06       721

    accuracy                           0.86      5244
   macro avg       0.67      0.51      0.49      5244
weighted avg       0.81      0.86      0.81      5244

======================= infrastructure_related ======================
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4907
           1       0.00      0.00      0.00       337

    accuracy                           0.93      5244
   macro avg       0.47      0.50      0.48      5244
weighted avg       0.88      0.93      0.90      5244

======================= transport ======================
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4993
           1       0.83      0.08      0.14       251

    accuracy                           0.95      5244
 

C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sander.berkers\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 6. Improve your model
Use grid search to find better parameters. 

In [64]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019913B78B80>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000019913B78B80>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [ ]:
# parameters = 

# cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [75]:
pickle.dump(pipeline, open('../models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.